In [19]:
import pandas as pd
df = pd.read_excel("Room_Details.xlsm")
df


/home/presto/Documents/BKW/.venv/lib/python3.12/site-packages/openpyxl/reader/workbook.py:118: UserWarning: Print area cannot be set to Defined name: 'Leistungsermittlung KLT_HZG'!$G:$BS.
  warn(f"Print area cannot be set to Defined name: {defn.value}.")


,ID,Unnamed: 1,Unnamed: 2,SC_Raum_Höhe,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 74,Unnamed: 75,BKW_MEP_HZG_Typ (3),BKW_MEP_HZG_Leistung (3),BKW_MEP_HZG_Anzahl (3),Unnamed: 79,Unnamed: 80,BKW_MEP_HZG_Summe,BKW_MEP_HZG_Schachtzuordnung,BKW_MEP_HZG_Kommentar
0,NaN,NaN,NaN,NaN,NaN,Leistungsermittlung mit Raumtypen - KLT / HZG,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,Raumdaten,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,Gebäudeteil,Geschoss,zusätzliche \nRaum-Nr.(1),zusätzliche \nRaum-Nr.(2),Raum-Nr.,...,NaN,NaN,Beheizung (3),NaN,NaN,NaN,NaN,Summe\nHeizleistung\nGesamt,Schacht-zuordnung\nHZG,Kommentar HZG
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
161,r000001866680_00,3930.0,14030.0,NaN,NaN,NaN,U1_OKRFB,NaN,NaN,DU 23,...,0,NaN,NaN,NaN,0,0,NaN,0,NaN,NaN
162,r000001866681_00,3930.0,11630.0,NaN,NaN,NaN,U1_OKRFB,NaN,NaN,DU 25,...,0,NaN,NaN,NaN,0,0,NaN,0,NaN,NaN
163,r000001866683_00,3930.0,69981.0,NaN,NaN,NaN,U1_OKRFB,NaN,NaN,DU 27,...,0,NaN,NaN,NaN,0,0,NaN,0,NaN,NaN
164,xxx,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
import pandas as pd

# Step 1: Load with correct encoding
df = pd.read_csv("Room_details.csv", header=None, encoding="latin1", low_memory=False)

# Step 2: Drop fully empty rows and columns
df.dropna(axis=0, how="all", inplace=True)
df.dropna(axis=1, how="all", inplace=True)

# Step 3: Set first non-empty header row as column names
df.columns = df.iloc[0].fillna("").astype(str).str.strip()
df = df[1:]

# Step 4: Clean column text
df.columns = (
    df.columns
    .str.replace(r"Unnamed:.*", "", regex=True)
    .str.replace("\\n", " ", regex=True)
    .str.replace(r"[^0-9a-zA-ZäöüÄÖÜß ]+", " ", regex=True)
    .str.strip()
    .str.replace(r"\\s+", "_", regex=True)
    .str.lower()
)

# Step 5: Define translation map (German → English)
translate_map = {
    "geschoss": "floor",
    "raum_nr": "room_no",
    "raum_bezeichnung": "room_name",
    "nummer_raumtyp": "room_type_no",
    "bezeichnung_raumtyp": "room_type_name",
    "fläche": "area_m2",
    "volumen": "volume_m3",
    "spez_kältebedarf": "specific_cooling_demand_m2",
    "spez_kältebedarf_pro_person": "specific_cooling_demand_person",
    "zusätzliche_innere_lasten": "additional_internal_loads_w",
    "gesamt_kühllast": "total_cooling_load_kw",
    "kühlung_1_typ": "cooling_type_1",
    "kühlleistung": "cooling_capacity_w",
    "anzahl": "cooling_count",
    "gesamt": "cooling_total_kw",
    "spez_wärmebedarf": "specific_heating_demand_m2",
    "gesamt_heizlast": "total_heating_load_kw",
    "beheizung_1_typ": "heating_type_1",
    "heizleistung": "heating_capacity_w",
    "heizleistung_anzahl": "heating_count",
    "heizleistung_gesamt": "heating_total_kw"
}

# Step 6: Apply translation and rename matching columns
cleaned_cols = [translate_map.get(col, col) for col in df.columns]
df.columns = cleaned_cols

# Step 7: Drop empty or duplicate columns
df = df.loc[:, df.columns != ""]
df = df.loc[:, ~df.columns.duplicated()]

# Step 8: Remove empty cells
df.replace(["", " ", " "], None, inplace=True)
df.dropna(how="all", inplace=True)

# Step 9: Save translation-ready clean dataset
df.to_csv("Room_details_ml_ready.csv", index=False, encoding="utf-8-sig")

print("✅ ML-ready file saved as 'Room_details_ml_ready.csv'")
print(f"Columns ({len(df.columns)}): {df.columns.tolist()[:20]}")



✅ ML-ready file saved as 'Room_details_ml_ready.csv'
Columns (51): ['id', 'sc raum höhe', 'bkw mep klt spez  kältebedarf', 'bkw mep klt spez  kältebedarf pro person', 'bkw mep klt zus  innere lasten', 'bkw mep klt kühllastberechnung extern', 'bkw mep klt kühllast gesamt', 'bkw mep klt kühlung typ  1', 'bkw mep klt kühlung leistung  1', 'bkw mep klt kühlung anzahl  1', 'bkw mep klt kühlung typ  2', 'bkw mep klt kühlung leistung  2', 'bkw mep klt kühlung anzahl  2', 'bkw mep klt kühlung typ  3', 'bkw mep klt kühlung leistung  3', 'bkw mep klt kühlung anzahl  3', 'bkw mep klt kühlung summe', 'bkw mep klt kühlbedarf typ  1', 'bkw mep klt kühlbedarf leistung  1', 'bkw mep klt kühlbedarf anzahl  1']


In [37]:
###ACTUAL CODE STARTS BELOW###############################################

import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier

# Load data
df = pd.read_csv("cleaned_room_details_2.csv", encoding="latin1")
df.columns = [col.strip().replace(" ", "_").replace("/", "_").replace(".", "").lower() for col in df.columns]

# Drop empty rows/columns
df.dropna(axis=0, how="all", inplace=True)
df.dropna(axis=1, how="all", inplace=True)

# Encoding for categorial
cat_cols = ['floor', 'room_no', 'room_name', 'room_type_name', 'cooling_type', 'heating_type']
for col in cat_cols:
    if col in df.columns:
        df[col] = LabelEncoder().fit_transform(df[col].astype(str))
df = df.drop(columns=["room_name", "room_type_name"])
# Prepare features/target
target = "room_type_no"
features = [col for col in df.columns if col != target]
X = df[features].apply(pd.to_numeric, errors="coerce").fillna(0)
y = df[target]

# Model and importances
model = RandomForestClassifier(random_state=42)
model.fit(X, y)
importances = pd.Series(model.feature_importances_, index=features)
top3 = importances.sort_values(ascending=False).head(3)
print("Top 3 user inputs:", top3.index.tolist())


Top 3 user inputs: ['volume_m3', 'area_m2', 'total_heating_load_kw']


In [39]:
##Most important features:Top 3 user inputs: ['volume_m3', 'area_m2', 'total_heating_load_kw']
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
import joblib

# 1. Load your data
df = pd.read_csv("cleaned_room_details_2.csv", encoding="latin1")
df.columns = [col.strip().replace(" ", "_").replace("/", "_").replace(".", "").lower() for col in df.columns]

# 2. Define features and target
features = ['volume_m3', 'area_m2', 'total_heating_load_kw']
target = 'room_type_no'

# 3. Convert features to numeric, drop missing
X = df[features].apply(pd.to_numeric, errors='coerce')
y = pd.to_numeric(df[target], errors='coerce')
mask = X.notna().all(axis=1) & y.notna()
X, y = X[mask], y[mask].astype(int)

# 4. Split data and train the model
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)
model = RandomForestClassifier(n_estimators=50, random_state=42)
model.fit(X_train, y_train)
print("Accuracy:", model.score(X_test, y_test))

# 5. Save the trained model
joblib.dump(model, "room_type_predictor.joblib")

# 6. Prediction function for new inputs
def predict_room_type(volume_m3, area_m2, total_heating_load_kw):
    loaded_model = joblib.load("room_type_predictor.joblib")
    inp = pd.DataFrame([[volume_m3, area_m2, total_heating_load_kw]], columns=features)
    return loaded_model.predict(inp)[0]

# Usage example
result = predict_room_type(362930, 97000, 388)
print("Predicted Room_Type_No:", result)



Accuracy: 0.7435897435897436
Predicted Room_Type_No: 5
